In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.pyplot import figure
import matplotlib
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
import pmdarima as pm
import warnings
import statsmodels
import datetime as dt
from sklearn.metrics import mean_squared_error
warnings.filterwarnings('ignore')

In [ ]:
#creating dataframes to test each varaible independently 

df = pd.read_csv('AllScorePerDay',index_col='dates')
df2 = pd.read_csv('AllPropPword',index_col='dates')
df3 = pd.read_csv('NegPerDay', index_col='dates')
df4 = pd.read_csv('NeuPerDay', index_col='dates')
df5 = pd.read_csv('PosPerDay', index_col='dates')
df6 = pd.read_csv('PropPosNeg', index_col='dates')
dfP = pd.read_csv('AllPropPword', index_col='dates')

In [ ]:
#creating dummy variables, both datasets had 'holiday ouliers' in the same dates, so no need for manual codification

df['holiday'] = 0
df['covid'] = 0

for index, score in enumerate(df['score']):
    if score >0.04:
        df['holiday'][index] = 1
    if score < -0.18:
        df['covid'][index] = 1


In [ ]:
#testing differentiation

df['diff']= df['0'].diff()
df = df.dropna()
df['diff'].plot()

In [ ]:
#checking dickey-fuller test for stationarity

ad_test = adfuller(df['diff'])

In [ ]:
#plotting autocorrelations and partial autocorrelations in stationary data

plot_acf(df['diff'],zero = False)
plot_pacf(df['diff'], zero = False)

In [ ]:
#testing best parameters for our ARIMA model

model_res = []

for p in range(7):
    for q in range(7):
        model = SARIMAX(train, order = (p,1,q))
        results = model.fit()
        model_res.append((p,q,results.aic, results.bic))

order_df = pd.DataFrame(model_res, 
                        columns=['p','q','AIC','BIC'])

print(order_df.sort_values('AIC'))
print(order_df.sort_values('BIC'))


In [ ]:
#creating a Dataframe to introduce exogenous variables in the ARIMA model, subituting lost values with mean

dfwords = pd.DataFrame()
dfwords['neg'] = df2['0']
dfwords['neu'] = df3['0']
dfwords['pos'] = df4['0']
dfwords['prop'] = df5['0']
dfwords['neg'] = dfwords['neg'].shift(1)
dfwords['neg'][0] = np.mean(dfwords['neg'])
dfwords['neu'] = dfwords['neu'].shift(1)
dfwords['neu'][0] = np.mean(dfwords['neu'])
dfwords['pos'] = dfwords['pos'].shift(1)
dfwords['pos'][0] = np.mean(dfwords['pos'])
dfwords['prop'] = dfwords['prop'].shift(1)
dfwords['prop'][0] = np.mean(dfwords['prop'])

df3 = pd.DataFrame()
df3['covid'] = df['covid']
df3['holiday'] = df['holiday']
dfP = dfP.shift(1)
dfP = dfP.fillna(np.mean(dfP))


In [ ]:
#main ARIMA model, residuals diagnostics and model decompositions 

model = ARIMA(df['score'], order=(0,1,3),seasonal_order=(1,0,0,7))
results = model.fit()

print(results.summary())
results.plot_diagnostics()
decomp = seasonal_decompose(train,period=7)
decomp.plot()
plt.show()

In [ ]:
#calculating by hand ARIMA predicitons as a sanity check

a = (results.resid[-1]* -0.4126)+(results.resid[-2] *-0.2524)+(results.resid[-3]*-0.1981)+(df['score'][360]*0.2706)
a + df['score'][365]

In [ ]:
#testing best parameters for our ARIMA model manually and with the auto_arima module 

model_res = []

for p in range(7):
    for q in range(7):
        model = SARIMAX(train, order = (p,1,q))
        results = model.fit()
        model_res.append((p,q,results.aic, results.bic))

order_df = pd.DataFrame(model_res, 
                        columns=['p','q','AIC','BIC'])

print(order_df.sort_values('AIC'))
print(order_df.sort_values('BIC'))

model1 = pm.auto_arima(df['0'], #time series
                      seasonal=True, # is the time series seasonal
                      max_p=6, # max value of p to test 
                      max_q=6, # max value of p to test
                      max_P=6, # max value of P to test 
                      max_Q=6, # max value of Q to test,
                      m=7,
                      information_criterion='aic', # used to select best mode
                      trace=True, # prints the information_criterion for each model it fits
                      error_action='ignore', # ignore orders that don't work
                      stepwise=True, # apply an intelligent order search
                      suppress_warnings=True) 

# Print model summary
print(model1.summary())

In [ ]:
#plotting

figure(figsize=(20,12), dpi=320)
plt.plot(arima_mean.index, arima_mean, label='forecasted',color='red')
plt.plot(arima_mean.index,df['score'])
plt.legend()

In [ ]:
#Cross-Validations testing

starts = [0,91,182,273]
rmse = []

for s in starts:
    pred = results.get_prediction(start=s,end=s+91,dynamic=False)
    pred_mean = pred.predicted_mean
    true_values = df['0'].iloc[s:s+92]
    pred_mean.plot()
    rmse.append(np.sqrt(mean_squared_error(true_values, pred_mean)))

np.mean(rmse)

In [ ]:
#Main mehtod to dinamaically predict one-step-ahead scores 'cheats' (uses coefficients generated at the end of the model), 
#this is the method used instead

pred = []
for s in range(366,731):
    exog = df3
    model = ARIMA(df['score'][:s], order=(0,1,3),seasonal_order=(1,0,0,7),exog=exog[:s])
    results = model.fit()
    exog = df3.iloc[s]
    pre = results.predict(start=s,exog=exog)
    pred.append(pre)

In [ ]:
#RMSE calculation

np.sqrt(mean_squared_error(df['score'][366:], pred))